# DAIS Demo - Field Technician Scheduler

This notebook demonstrates the field technician scheduling system for the self-healing networks project. It shows how the system generates realistic technician routes and manages daily scheduling.

## Overview
- **Purpose**: Demonstrate field technician scheduling capabilities
- **Features**: Route generation, scheduling simulation, GPS tracking
- **Integration**: Works with the main triage and routing system


In [ ]:
## Step 1: Install Dependencies

Install the Faker library for generating realistic test data including names, addresses, and other synthetic information for the field technician simulation.


In [0]:
%pip install faker
dbutils.library.restartPython()

In [ ]:
## Step 2: Generate Field Technician Routes

Generate realistic daily routes for field technicians including:
- Random scheduling (some technicians off work)
- Variable number of stops per route
- GPS coordinates based on tower locations
- Realistic route patterns with small coordinate variations


In [0]:
from datetime import datetime
import random
from pyspark.sql import Row
from faker import Faker
fake = Faker()

sf_sampled = spark.table("telco_demos.self_healing_networks.sf_towers")
field_technicians = spark.table("telco_demos.self_healing_networks.field_technicians")
route_data = []

lons = sf_sampled.select('lon').toPandas()['lon'].tolist()
lats = sf_sampled.select('lat').toPandas()['lat'].tolist()
for tech in field_technicians.collect():
    if random.randint(1, 7) <= 2:  # 2/7 chance of being off work
        route_data.append(Row(
            guid=tech.tech_id,
            name=tech.name,
            lat=None,
            lon=None,
            stop_number=None,
            random_text="not scheduled",
            date=datetime.today().strftime('%Y-%m-%d'),
        ))
    else:
        num_stops = random.randint(1, 15)
        base_lat = random.choice(lats)
        base_lon = lons[lats.index(base_lat)]
        for stop_num in range(1, num_stops + 1):
            route_data.append(Row(
                guid=tech.tech_id,
                name=tech.name,
                lat=base_lat + random.uniform(-0.01, 0.01),
                lon=base_lon + random.uniform(-0.01, 0.01),
                stop_number=stop_num,
                random_text=fake.text(max_nb_chars=50),
                date=datetime.today().strftime('%Y-%m-%d'),
            ))

routes_df = spark.createDataFrame(route_data)
routes_df.write.mode("append").saveAsTable("telco_demos.self_healing_networks.fieldtech_route")
display(routes_df)